In [1]:
from webull import paper_webull, endpoints # for real money trading, just import 'webull' instead
from webull.streamconn import StreamConn
from day_trade import login_webull
from torchvision.models import resnet50
import torch
import sched
import matplotlib.pyplot as plt
import mplfinance as fplt
import time
from PIL import Image
from torchvision import datasets, transforms
import json
import pandas as pd

In [2]:
torch.cuda.get_device_name(0), torch.cuda.is_available()

('GeForce GTX 1070 Ti', True)

In [4]:
wb = login_webull('mcian91@email.com')

In [5]:
loginInfo = json.load(open('token.txt'))

In [3]:
model_res = resnet50(pretrained=False, num_classes=3)
model_res.load_state_dict(torch.load('./dukascopy/model_res50.pth'))

<All keys matched successfully>

In [6]:
if torch.cuda.is_available():
    model_res = model_res.cuda()

In [7]:
hist = None
support = 0
resistance = 0
enteredTrade = False
s = sched.scheduler(time.time, time.sleep)

In [8]:
TRANSFORM_IMG = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.ToTensor()
    ])

In [9]:
#Draw Chart
def drawChart(hist, update):
    global support
    global resistance
    global symbol
    try :
        mc = fplt.make_marketcolors(
            up='tab:blue',down='tab:red',
            edge='black',
            wick={'up':'blue','down':'red'},
            volume='green', alpha=1.0
        )
        s  = fplt.make_mpf_style(marketcolors=mc, mavcolors=["yellow","orange",'brown'])
        image_file = './live_graphs/'+ symbol + '_' + hist.index[-1].isoformat().replace(':', '-') +'.png'
        fig = fplt.plot(
            hist,
            type='candle',
            style=s,
            volume=True,
            axisoff=True,
            mav=(5,8,13),
            figsize=(3.2,3.2),
            xlim=(12.5, 44.5),
            savefig=image_file,
            tight_layout=True
        )
        return image_file
    except Exception as e:
        print(e)
#On Bar Update
def run(sc):
    global hist
    global enteredTrade
    global symbol
    global s
    hist = pd.DataFrame(hist)
    try:
        #Update chart with new data
        hist = wb.get_bars(stock=symbol.upper(), interval='m1', count=44, extendTrading=0)
        hist = pd.DataFrame(hist)
        #call this method again every minute for new price changes
        image_file = drawChart(hist, True)
        img = Image.open(image_file).convert("RGB")
        outputs = model_res(TRANSFORM_IMG(img).reshape((1,3,256,256)).cuda())
        probs = torch.nn.functional.softmax(outputs.data)[0].tolist()
        _, preds = torch.max(outputs.data, 1)
        label = preds[0].item()
        if not enteredTrade and label == 0:
            order = wb.place_order(stock=symbol.upper(), action='BUY', orderType='MKT', enforce='DAY', quant=1)
            print(order)
            enteredTrade = True
        elif enteredTrade and label == 2:
            order = wb.place_order(stock=symbol.upper(), action='SELL', orderType='MKT', enforce='DAY', quant=1)
            print(order)
            enteredTrade = False
    except Exception as e:
        print(str(e))
    s.enter(60, 1, run, (sc,))
    plt.pause(60)

In [12]:
conn = StreamConn(debug_flg=False)
if not loginInfo['accessToken'] is None and len(loginInfo['accessToken']) > 1:
    conn.connect(loginInfo['uuid'], access_token=loginInfo['accessToken'])
else:
    conn.connect(wb.did)
#Initiate our scheduler so we can keep checking every minute for new price changes
s.enter(1, 1, run, (s,))
def get_data():
    global symbol
    global hist
    try:
        #Symbol to trade
        symbol = 'RKT'
        hist = wb.get_bars(stock=symbol.upper(), interval='m1', count=44, extendTrading=0)
        hist = pd.DataFrame(hist)
        s.run()
    except KeyboardInterrupt:
        print('Interrupted')
        pass
    except Exception as e:
        print(e)
        get_data()
get_data()

say hello
Interrupted
